In [ ]:
import numpy as np
from scipy.signal import convolve

def compute_baseline(movie, f_noise_sigma, mean_window_size, num_iterations):
    N, M, T = movie.shape
    movie = movie.astype(np.float32)
    flags = np.ones((N, M), dtype = bool)

    w_sizes = np.clip(np.arange(T) + (mean_window_size + 1) // 2, 0, T) \
            - np.clip(np.arange(T) - mean_window_size // 2, 0, T)
    w_sizes = w_sizes.astype(np.float32)
    kernel = np.ones((mean_window_size), dtype = np.float32)[np.newaxis]
    sqrtT = (T / (T - 1))**(0.5)

    if N > M:
        for elem in range(N):
            init = movie[elem].copy()
            mark_flags = flags[elem]
            for _ in range(num_iterations):
                init[mark_flags] = convolve(movie[elem], kernel, mode = 'same', method = 'fft')[mark_flags] / w_sizes
                np.minimum(init, movie[elem], out = movie[elem])
                diff = (init - movie[elem])**2
                D = sqrtT*(np.mean(diff, axis = 1))**(0.5)
                mark_flags &= (D >= f_noise_sigma[elem])
            movie[elem] = init
    else:
        for elem in range(M):
            init = movie[:, elem].copy()
            mark_flags = flags[:, elem]
            for _ in range(num_iterations):
                init = convolve(movie[:, elem], kernel, mode = 'same', method = 'fft') / w_sizes
                np.minimum(init, movie[:, elem], out = movie[:, elem])
                diff = (init - movie[:, elem])**2
                D = sqrtT*(np.mean(diff, axis = 1))**(0.5)
                mark_flags &= (D >= f_noise_sigma[:, elem])
            movie[:, elem] = init
    return movie

if __name__ == "__main__":
    movie = np.array(
    [[[ 3, -5,  1, -1, -3,  0], [ 4, 4, 1, -5, -3, -1]],
     [[-2, -4, -2,  0,  2, -3], [-2, 2, 2,  3, -2, 0]]], dtype=np.float64)

    f_noise_sigma = np.array([[1, 1], [1, 0]], dtype=np.float64)
    mean_window_size = 6
    num_iterations = 4
    movie = compute_baseline(movie, f_noise_sigma, mean_window_size, num_iterations)
    print(movie)

[[[-3.62142   -3.3603358 -3.288269  -3.0881984 -3.0939863 -2.617483 ]
  [-0.8351113 -1.8763336 -2.1010666 -2.2179723 -2.6909003 -3.0786254]]

 [[-3.2898026 -3.1761017 -3.1057258 -3.0881052 -3.0922446 -2.8653057]
  [-1.1776251 -1.080823  -1.2646582 -1.1717986 -1.0061584 -1.1003542]]]


In [ ]:
import numpy as np
from scipy.signal import convolve

def compute_baseline(movie, f_noise_sigma, mean_window_size, num_iterations):
    N, M, T = movie.shape
    movie = movie.astype(np.float32)
    w_sizes = np.clip(np.arange(T) + (mean_window_size + 1) // 2, 0, T) \
            - np.clip(np.arange(T) - mean_window_size // 2, 0, T)
    w_sizes = w_sizes.astype(np.float32)
    kernel = np.ones((mean_window_size), dtype = np.float32)[np.newaxis, np.newaxis]

    if N > 2:
        if N % 2 == 0:
            movie = movie.reshape(2, N // 2, M, T)
            f_noise_sigma = f_noise_sigma.reshape(2, N // 2, M)
        
        else:
            movie0 = movie[:N // 2, :, :]
            movie1 = movie[N // 2:, :, :]
            movie = [movie0, movie1]

            f_noise_sigma0 = f_noise_sigma[:N // 2, :]
            f_noise_sigma1 = f_noise_sigma[N // 2:, :]
            f_noise_sigma = [f_noise_sigma0, f_noise_sigma1]

            flags0 = np.ones((N // 2, M), dtype = bool)
            flags1 = np.ones((N // 2 + 1, M), dtype = bool)

            flags = [flags0, flags1]

        for elem in range(2):
            init = movie[elem].copy()
            for _ in range(num_iterations):
                init[flags[elem]] = convolve(movie[elem], kernel, mode = 'same', method = 'fft')[flags[elem]] / w_sizes
                movie[elem] = np.minimum(init, movie[elem])

                D = np.sqrt(np.mean((init - movie[elem]) ** 2, axis = 2) * T / (T - 1))
                flags[elem] &= (D >= f_noise_sigma[elem])
            movie[elem] = init
        
        if N % 2 == 0:
            return np.array(movie).reshape(N, M, T)
        else:
            arr = np.zeros([N, M, T])
            arr[:N // 2, :, :] = movie[0]
            arr[N // 2:, :, :] = movie[1]
            return arr
    
    else:
        flags = np.ones((N, M), dtype = bool)
        init = movie.copy()
        for _ in range(num_iterations):
            init[flags] = convolve(movie, kernel, mode = 'same', method = 'fft')[flags] / w_sizes
            movie = np.minimum(init, movie)

            D = np.sqrt(np.mean((init - movie) ** 2, axis = 2) * T / (T - 1))
            flags &= (D >= f_noise_sigma)
        movie = init
        return movie


if __name__ == "__main__":
    np.random.seed(4)
    movie = np.random.random((121, 1, 1000))

    f_noise_sigma = np.random.random((121, 1))
    mean_window_size = 1000
    num_iterations = 10
    movie = compute_baseline(movie, f_noise_sigma, mean_window_size, num_iterations)
    print(movie)

In [ ]:
import numpy as np

# Создаем исходный массив размера (10, 10, 100)
array = np.array(
                [[[ 3, -5,  1, -1, -3,  0], [ 4, 4, 1, -5, -3, -1]],
                 [[-2, -4, -2,  0,  2, -3], [-2, 2, 2,  3, -2, 0]]], dtype=np.float64)

# Изменяем форму массива на (4, 5, 5, 100), чтобы получить 4 массива (5, 5, 100)
reshaped_array = array.reshape(4, 1, 1, 6)

# Преобразуем его в список
result_list = [reshaped_array[i] for i in range(reshaped_array.shape[0])]

# Проверяем результат
print(np.round(array, 1))  # Должно вывести (5, 5, 100)
print(np.round(reshaped_array, 1))  # Должно вывести 4

In [ ]:
import numpy as np
from scipy.signal import convolve

def compute_baseline(movie, f_noise_sigma, mean_window_size, num_iterations):
    N, M, T = movie.shape
    movie = movie.astype(np.float32)

    flags = np.ones((N, M), dtype = bool)

    left = np.floor(mean_window_size / 2, dtype = np.float32)
    right = np.ceil(mean_window_size / 2, dtype = np.float32)

    w_sizes = np.minimum(T, np.arange(T) + right) - np.maximum(0, np.arange(T) - left)
    w_sizes = w_sizes.astype(np.float32)
    kernel = np.ones((mean_window_size,), dtype = np.float32)

    init = movie
    for _ in range(num_iterations):
        if not np.any(flags):
            break
        swx = init[~flags]
        init = convolve(movie, kernel[np.newaxis, np.newaxis], mode='same') / w_sizes
        init[~flags] = swx
        movie = np.minimum(init, movie)
        D = np.sqrt(np.mean((init - movie) ** 2, axis=2) * T / (T - 1))
        flags = flags & (D >= f_noise_sigma)
    return init


if __name__ == "__main__":
    np.random.seed(4)
    movie = np.random.random((100, 100, 1000))

    f_noise_sigma = np.random.random((100, 100))
    mean_window_size = 1000
    num_iterations = 10
    movie = compute_baseline(movie, f_noise_sigma, mean_window_size, num_iterations)
    print(movie)

In [ ]:
import numpy as np
from scipy.signal import convolve

def compute_baseline(movie, f_noise_sigma, mean_window_size, num_iterations):
    N, M, T = movie.shape
    movie = movie.astype(np.float32)

    flags = np.ones((N, M), dtype = bool)

    left = np.floor(mean_window_size / 2, dtype = np.float32)
    right = np.ceil(mean_window_size / 2, dtype = np.float32)

    w_sizes = np.minimum(T, np.arange(T) + right) - np.maximum(0, np.arange(T) - left)
    w_sizes = w_sizes.astype(np.float32)
    kernel = np.ones((mean_window_size,), dtype = np.float32)

    init = movie
    for _ in range(num_iterations):
        if not np.any(flags):
            break
        swx = init[~flags]
        init = convolve(movie, kernel[np.newaxis, np.newaxis], mode='same') / w_sizes
        init[~flags] = swx
        movie = np.minimum(init, movie)
        D = np.sqrt(np.mean((init - movie) ** 2, axis=2) * T / (T - 1))
        flags = flags & (D >= f_noise_sigma)
    return init


if __name__ == "__main__":
    np.random.seed(4)
    movie = np.random.random((100, 100, 1000))

    f_noise_sigma = np.random.random((100, 100))
    mean_window_size = 1000
    num_iterations = 10
    movie = compute_baseline(movie, f_noise_sigma, mean_window_size, num_iterations)
    print(movie)

In [ ]:
import numpy as np
from scipy.signal import convolve

def compute_baseline(movie, f_noise_sigma, mean_window_size, num_iterations):
    N, M, T = movie.shape
    movie = movie.astype(np.float32)
    flags = np.ones((N, M), dtype = bool)

    w_sizes = np.clip(np.arange(T) + (mean_window_size + 1) // 2, 0, T) \
            - np.clip(np.arange(T) - mean_window_size // 2, 0, T)
    w_sizes = w_sizes.astype(np.float32)
    kernel = np.ones((mean_window_size), dtype = np.float32)[np.newaxis, np.newaxis]

    init = movie.copy()
    for _ in range(num_iterations):
        init[flags] = convolve(movie, kernel, mode = 'same', method = 'direct')[flags] / w_sizes
        movie = np.minimum(init, movie)
        D = np.sqrt(np.mean((init - movie) ** 2, axis = 2) * T / (T - 1))
        flags &= (D >= f_noise_sigma)
    return init


if __name__ == "__main__":
    movie = np.array(
    [[[ 3, -5,  1, -1, -3,  0], [ 4, 4, 1, -5, -3, -1]],
     [[-2, -4, -2,  0,  2, -3], [-2, 2, 2,  3, -2, 0]]], dtype=np.float64)

    f_noise_sigma = np.array([[1, 1], [1, 0]], dtype=np.float64)
    mean_window_size = 6
    num_iterations = 4
    movie = compute_baseline(movie, f_noise_sigma, mean_window_size, num_iterations)
    print(movie)

In [42]:
import numpy as np
from scipy.signal import convolve


def compute_baseline(movie, f_noise_sigma, mean_window_size, num_iterations):
    N, M, T = movie.shape
    movie = movie.astype(np.float32)
    flags = np.ones((N, M), dtype = bool)

    w_sizes = np.clip(np.arange(T) + (mean_window_size + 1) // 2, 0, T) \
            - np.clip(np.arange(T) - mean_window_size // 2, 0, T)
    w_sizes = w_sizes.astype(np.float32)
    kernel = np.ones((mean_window_size), dtype = np.float32)[np.newaxis, np.newaxis]

    init = movie.copy()
    for _ in range(num_iterations):
        init[flags] = convolve(movie, kernel, mode = 'same', method = 'fft')[flags] / w_sizes
        np.minimum(init, movie, out = movie)
        
        D = np.sqrt(np.mean((init - movie) ** 2, axis = 2) * T / (T - 1))
        flags &= (D >= f_noise_sigma)
    return init

if __name__ == "__main__":
    movie = np.array(
    [[[ 3, -5,  1, -1, -3,  0], [ 4, 4, 1, -5, -3, -1]],
     [[-2, -4, -2,  0,  2, -3], [-2, 2, 2,  3, -2, 0]]], dtype=np.float64)

    f_noise_sigma = np.array([[1, 1], [1, 0]], dtype=np.float64)
    mean_window_size = 6
    num_iterations = 4
    movie = compute_baseline(movie, f_noise_sigma, mean_window_size, num_iterations)
    print(movie)

[[[-3.62142   -3.3603358 -3.288269  -3.0881984 -3.0939863 -2.617483 ]
  [-0.8351113 -1.8763336 -2.1010666 -2.2179723 -2.6909003 -3.0786254]]

 [[-2.888889  -2.541667  -2.3133335 -2.427778  -2.38      -1.9750001]
  [-1.1776251 -1.080823  -1.2646582 -1.1717986 -1.0061584 -1.1003542]]]
